# 📖 TABLE OF CONTENTS

- [1. Introduction]()
- [2. Installing Dependencies]()
- [3. Mount Google Drive & Load API Keys]()
- [4. Approach 1: Alternating Human/AI Messages]()
- [5. Approach 2: Few-Shot Prompting]()
- [6. Approach 3: Example Selectors]()
  - [1. Using `LengthBasedExampleSelector`]()
  - [2. Using `SemanticSimilarityExampleSelector`]()
- [7. Conclusion]()

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 1. Introduction

In this lesson, we'll explore how few-shot prompts and example selectors can enhance the performance of language models in LangChain. Implementing **Few-shot prompting** and **Example selection** in LangChain can be achieved through various methods. We'll discuss three distinct approaches, examining their advantages and disadvantages to help you make the most of your language model.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 2. Installing Dependencies

In [1]:
!pip3 install langchain==0.0.208 deeplake openai==0.27.8 python-dotenv tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.9/607.9 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.4 MB/s eta 0:00:00


![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 3. Mount Google Drive & Load API Keys

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


All the API Keys are stored in file "llm_env". It's contents are as below:

ACTIVELOOP_TOKEN=<[Your Activeloop API Key](https://app.activeloop.ai/register)>

OPENAI_API_KEY=<[Your OpenAI API Key](https://platform.openai.com/)>

GOOGLE_API_KEY=<[Your Google API Key](https://console.cloud.google.com/apis/credentials)>

GOOGLE_CSE_ID=<[Your Google Custom Search Engine ID](https://programmablesearchengine.google.com/controlpanel/create)>

HUGGINGFACEHUB_API_TOKEN=<[Your Hugging Face Access Token](https://huggingface.co/settings/tokens)>

In [2]:
from dotenv import load_dotenv

# Load API Keys for Deep Lake Vector Database, OpenAI, Google & Hugging Face
load_dotenv('/content/drive/MyDrive/ancilcleetus-github/llm_env')

True

In [ ]:
import os

print(f"os.environ['ACTIVELOOP_TOKEN']: \n{os.environ['ACTIVELOOP_TOKEN']}")
print(f"os.environ['OPENAI_API_KEY']: \n{os.environ['OPENAI_API_KEY']}")
print(f"os.environ['GOOGLE_API_KEY']: \n{os.environ['GOOGLE_API_KEY']}")
print(f"os.environ['GOOGLE_CSE_ID']: \n{os.environ['GOOGLE_CSE_ID']}")
print(f"os.environ['HUGGINGFACEHUB_API_TOKEN']: \n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 4. Approach 1: Alternating Human/AI Messages

In this strategy, few-shot prompting utilizes alternating human and AI messages. This technique can be especially beneficial for chat-oriented applications since the language model must comprehend the conversational context and provide appropriate responses.

While this approach effectively handles conversation context and is easy to implement for chat-based applications, it lacks flexibility for other application types and is limited to chat-based models. However, we can use alternating human/AI messages to create a chat prompt that translates English into pirate language. The code snippet below demonstrates this approach.

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Initialize Chat Model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template="You are a helpful assistant that translates English to pirate language."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run("I love programming.")

"I love programmin', matey."

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 5. Approach 2: Few-Shot Prompting

Few-shot prompting can lead to improved output quality because the model can learn the task better by observing the examples. However, the increased token usage may worsen the results if the examples are not well chosen or are misleading.

This approach involves using the `FewShotPromptTemplate` class, which takes in a `PromptTemplate` and a list of a few shot examples. The class formats the prompt template with a few shot examples, which helps the language model generate a better response. We can streamline this process by utilizing LangChain's `FewShotPromptTemplate` to structure the approach:

In [5]:
from langchain import PromptTemplate, FewShotPromptTemplate

# Create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# Create an example template
example_template = """
User: {query}
AI: {answer}
"""

# Create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# Now break our previous prompt into a prefix and suffix
# Prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""

# Suffix is our user input and output indicator
suffix = """
User: {query}
AI: """

# Create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the secret to happiness?")

'The secret to happiness is a good internet connection and a full battery!'

This method allows for better control over example formatting and is suitable for diverse applications, but it demands the manual creation of few-shot examples and can be less efficient with a large number of examples.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 6. Approach 3: Example Selectors

Example selectors can be used to provide a few-shot learning experience. The primary goal of few-shot learning is to learn a similarity function that maps the similarities between classes in the support and query sets. In this context, an example selector can be designed to choose a set of relevant examples that are representative of the desired output.

The `ExampleSelector` is used to select a subset of examples that will be most informative for the language model. This helps in generating a prompt that is more likely to generate a good response. Also, the `LengthBasedExampleSelector` is useful when you're concerned about the length of the context window. It selects fewer examples for longer queries and more examples for shorter queries.

## 1. Using `LengthBasedExampleSelector`

In [6]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Define our examples
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

# Define the example_prompt
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template
)

# Create an instance of LengthBasedExampleSelector
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)

# Create a FewShotPromptTemplate
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Generate a prompt using the format method
print(dynamic_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad



Word: tall
Antonym: short



Word: energetic
Antonym: lethargic



Word: sunny
Antonym: gloomy


Word: big
Antonym:


This method is effective for managing a large number of examples. It offers customization through various selectors, but it involves manual creation and selection of examples, which might not be ideal for every application type.

## 2. Using `SemanticSimilarityExampleSelector`

Example of employing LangChain's `SemanticSimilarityExampleSelector` for selecting examples based on their semantic resemblance to the input is given below. This illustration showcases the process of creating an `ExampleSelector`, generating a prompt using a few-shot approach:

In [10]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Create a PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

# Create Deep Lake dataset
# TODO: use your organization id here.  (by default, org id is your username)
my_activeloop_org_id = "ancilcleetus"
my_activeloop_dataset_name = "langchain_course_fewshot_selector_dataset"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

# Embedding function
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:",
    input_variables=["temperature"],
)

# Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) # Test with a new input after adding the example

Your Deep Lake dataset has been successfully created!


Deep Lake Dataset in ./deeplake/ already exists, loading from the storage


Creating 5 embeddings in 1 batches of size 5:: 100%|██████████| 1/1 [00:00<00:00,  3.44it/s]


Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (17, 1536)  float32   None   
    id        text      (17, 1)      str     None   
 metadata     json      (17, 1)      str     None   
   text       text      (17, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:00<00:00,  5.91it/s]

Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (18, 1536)  float32   None   
    id        text      (18, 1)      str     None   
 metadata     json      (18, 1)      str     None   
   text       text      (18, 1)      str     None   


Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:


Keep in mind that the `SemanticSimilarityExampleSelector` uses the Deep Lake vector store and `OpenAIEmbeddings` to measure semantic similarity. It stores the samples on the database in the cloud, and retrieves similar samples.

We created a `PromptTemplate` and defined several examples of temperature conversions. Next, we instantiated the `SemanticSimilarityExampleSelector` and created a `FewShotPromptTemplate` with the `selector`, `example_prompt`, and appropriate `prefix` and `suffix`.

Using `SemanticSimilarityExampleSelector` and `FewShotPromptTemplate` , we enabled the creation of versatile prompts tailored to specific tasks or domains, like temperature conversion in this case. These tools provide a customizable and adaptable solution for generating prompts that can be used with language models to achieve a wide range of tasks.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 7. Conclusion

To conclude, the utility of alternating human/AI interactions proves beneficial for chat-oriented applications, and the versatility offered by employing few-shot examples within a prompt template and selecting examples for the same extends its applicability across a broader spectrum of use cases. These methods necessitate a higher degree of manual intervention, as they require careful crafting and selection of apt examples. While these methods promise enhanced customization, they also underscore the importance of striking a balance between automation and manual input for optimal results.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)